# Upload course to Canvas

Het doel van onderstaande code is om alle informatie over deze course te onderhouden in het mapje waar dit notebook instaat, maar dit eenvoudig te laten synchroniseren met Canvas. Voor de docent zou dit moeten leiden tot één plek waar overzichtelijk alle informatie die onderhouden moet worden staat. Tegelijkertijd zou dit voor studenten moeten leiden tot eenzelfde overzichtelijke plek op Canvas, vanwaar zij alle course content kunnen zien.

Voor dit om (automatisch) te werken moeten de volgende mapjes bestaan:
* release
    * Per week met een assignment staat hier een mapje genaamd `AssignmentWeekX` (waar X een getal is van 1 t/m 9). Hierin staat één file wederom genaamd `AssignmentWeekX`. Deze file wordt op Canvas gezet.
* markdown
* data
    * Per week met een assignment staat hier een mapje genaamd `WeekX` (waar X een getal is van 1 t/m 9). In dit mapje staan alle files die bij een opdracht van een week-assignment horen.
    
    
TO-DO:

* Delete files which aren't used

In [1]:
%matplotlib inline

from __future__ import print_function

import csv
import os
import re
import shutil

import pypandoc
from traitlets.config import Config
from canvasapi import Canvas
from tqdm import tqdm, tqdm_notebook  # Progress bar

In [2]:
canvas_id = 9999  # Course id on canvas
API_URL = "https://canvas.uva.nl"
API_KEY = ""
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(canvas_id)

In [3]:
# Upload syllabus
html = pypandoc.convert_file('markdown/syllabus.md', 'html')
course.update(course={'syllabus_body':html,'public_syllabus':True})

'Data Science'

In [4]:
def upload_data_files(module, week, folder):
    if week in os.listdir('data/'):
        # Get all files which need to be uploaded
        for filename in os.listdir('data/%s' % week):

            # Delete file if it already exists
            files_in_folder = {
                file.filename: file
                for file in folder.get_files()
            }
            if filename in files_in_folder.keys():
                files_in_folder[filename].delete()

            # Upload file
            folder.upload('data/%s/%s' % (week, filename))
            # Get file_id
            file_id = [
                file.id for file in folder.get_files()
                if file.filename == filename
            ][0]
            list_of_items = [item.title for item in module.get_module_items()]
            if 'Databestand voor Assignment: %s' % filename not in list_of_items:
                module.create_module_item(
                    module_item={
                        'type': 'File',
                        'title': 'Databestand voor Assignment: %s' % filename,
                        'content_id': file_id,
                        'indent': 2
                    })


def upload_assignment_file(module, week, folder):
    if 'Assignment%s' % week in os.listdir('release/'):
        # Delete file if it already exists
        files_in_folder = {file.filename: file for file in folder.get_files()}
        if 'Assignment%s' % week in files_in_folder.keys():
            files_in_folder['Assignment%s' % week].delete()

        folder.upload('release/Assignment%s/Assignment%s.ipynb' % (week, week))
        file_id = [
            file.id for file in folder.get_files()
            if file.filename == 'Assignment%s.ipynb' % week
        ][0]
        # Check if file is module-item
        if 'Assignment%s' % week not in [
                item.title for item in module.get_module_items()
        ]:
            # Add assignment as module item
            module.create_module_item(
                module_item={
                    'type': 'File',
                    'title': 'Assignment%s' % week,
                    'content_id': file_id,
                    'indent': 1
                })


def create_inleverveld(week, module, course):
    list_of_items = [item.title for item in module.get_module_items()]
    if 'Assignment%s' % week in os.listdir('release/'):
        if 'Inleverveld %s' % week not in list_of_items:
            assignmentdict = {
                assignment.name: assignment.id
                for assignment in course.get_assignments()
            }

            # If Assignment does not exist, create assignment
            if 'Assignment%s' % week not in assignmentdict.keys():
                course.create_assignment(
                    assignment={'name': 'Assignment%s' % week})
                assignmentdict = {
                    assignment.name: assignment.id
                    for assignment in course.get_assignments()
                }

            # Add module item
            module.create_module_item(
                module_item={
                    'type': 'Assignment',
                    'title': 'Inleverveld %s' % week,
                    'content_id': assignmentdict['Assignment%s' % week],
                    'position': 2
                })


def create_module(week):
    modulesdict = {module.name: module for module in course.get_modules()}
    if week not in modulesdict.keys():
        course.create_module(module={'name': week, 'position': int(week[-1])})
        modulesdict = {module.name: module for module in course.get_modules()}
        modulesdict[week].edit(module={'published': True})
    return modulesdict[week]


def create_week_overview(week, module):
    pages = {page.title: page for page in course.get_pages()}
    if week not in pages.keys():
        # Create page
        course.create_page(
            wiki_page={
                'title': week,
                'body': pypandoc.convert_file('markdown/%s.md' % week, 'html'),
                'editing_roles': 'teachers'
            })
        # Select page and make it a module-item
        pages = {page.title: page for page in course.get_pages()}
        module.create_module_item(
            module_item={
                'type': 'Page',
                'title': 'Overzicht %s' % week,
                'content_id': pages[week].page_id,
                'page_url': week.lower(),
                'position': 1
            })
        # Select module-item and publish
        item = {item.title: item
                for item in module.get_module_items()}['Overzicht %s' % week]
        item.edit(module_item={'published': True})
    else:
        # Update page if already exists
        page = pages[week]
        page.edit(
            wiki_page={
                'body': pypandoc.convert_file('markdown/%s.md' % week, 'html')
            })


def create_folder(week, course, basefolder):
    dict_of_folders = {f.name: f for f in course.get_folders()}
    if week not in dict_of_folders.keys():
        return basefolder.create_folder(week)
    else:
        return dict_of_folders[week]

In [5]:
# Get Canvas base folder, to place other files in
basefolder = [f for f in course.get_folders() if f.name == 'course files'][0]

for n in tqdm_notebook(range(1, 9)):

    week = "Week%s" % n

    # Create module for each week if it doesn't exist
    module = create_module(week)

    # Create course plan page for week as page and module item
    create_week_overview(week, module)

    # Create canvas folder to place files in
    folder = create_folder(week, course, basefolder)

    create_inleverveld(week, module, course)

    upload_assignment_file(module, week, folder)

    upload_data_files(module, week, folder)